This module focuses on processig __FINANCIAL OHLCV__ data:
    1. Calculate the data for actual hourly/daily data
    2. Resave the file in correct format
    3. Write functions to automate these processes 
        ***getDeltaSeries() => replaceCseries()
    4  Explore merging the pumped time with actual pumped time

In [1]:
import os, sys
import pandas as pd 
import numpy as np
import requests 
from datetime import datetime, date, time, timedelta
import pytz
import matplotlib.pyplot as plt
import matplotlib.dates as mdate
from mpl_finance import candlestick_ohlc
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.ticker as mticker
pd.set_option('display.max_columns', 999)
%load_ext autoreload
%matplotlib inline

In [2]:
print(os.getcwd())

/Users/hn/Documents/USC Others/ISI Research/PumpCoin/CrytoCompare/All_Pump_Analysis/Before_Announcement_Analysis


# PART 1: TRANSFORM DATA INTO TRUE TIME SERIES

These OHLCV data is already in time series format. So the only pre-procssing to do is to convert them into satoshi.
** Reuse the code in previous analysis for consitency?

** There are 292 coins at the 3 exchanges Yobitt/Bittrex/Binance

__CONVERASION RULE__:

* 1 BTC = 10E8 Satoshi
* Smoothing : add 1 when conversion is 0

In [33]:
t = pd.read_pickle("../../Coin_Data/Daily/Daily_FTM_at_Binance.pkl")
t.head()

,time,close,high,low,open,volumefrom,volumeto,exchange,date,date_utc
0,1518048000,0.0,0.0,0.0,0.0,0,0.0,Binance,2018-02-07 16:00:00,2018-02-08 00:00:00+00:00
1,1518134400,0.0,0.0,0.0,0.0,0,0.0,Binance,2018-02-08 16:00:00,2018-02-09 00:00:00+00:00
2,1518220800,0.0,0.0,0.0,0.0,0,0.0,Binance,2018-02-09 16:00:00,2018-02-10 00:00:00+00:00
3,1518307200,0.0,0.0,0.0,0.0,0,0.0,Binance,2018-02-10 16:00:00,2018-02-11 00:00:00+00:00
4,1518393600,0.0,0.0,0.0,0.0,0,0.0,Binance,2018-02-11 16:00:00,2018-02-12 00:00:00+00:00


# DAILY

In [39]:
dir_prefix = "../../Coin_Data/Daily"
cnt = 0
fin_vars = ['open','high','low','close','volumefrom','volumeto']
exclude_vars = ['time','exchange','date','date_utc']
for filename in os.listdir(dir_prefix): 
    if 'at' in filename and '.pkl'  in filename:
        cnt+=1
        d = pd.read_pickle(dir_prefix + '/'+filename)
#         print(d.loc[2,fin_vars])
        
        #Convert financial features into Satoshi
        for var in fin_vars:
            d[var] = d[var].apply(lambda x: x*1e8 if x >= 1e-10 else x*1e8 + 1)
#         print(d.loc[2,fin_vars], "\n")
        
        #Save the files in the processed dataset
        pd.to_pickle(d, "../../Coin_Data/Daily/Processed_Data/Sat_"+filename)
    

## Verify the structure of processed data

In [41]:
test = pd.read_pickle("../../Coin_Data/Daily/Processed_Data/Sat_Daily_ADX_at_Binance.pkl")

In [42]:
test.head()

,time,close,high,low,open,volumefrom,volumeto,exchange,date,date_utc
0,1518048000,13920.0,14110.0,13890.0,13940.0,5.772600e+12,8.078000e+08,Binance,2018-02-07 16:00:00,2018-02-08 00:00:00+00:00
1,1518134400,14060.0,14340.0,13690.0,13920.0,2.738860e+13,3.853000e+09,Binance,2018-02-08 16:00:00,2018-02-09 00:00:00+00:00
2,1518220800,17200.0,17370.0,13890.0,14060.0,5.537624e+14,8.662700e+10,Binance,2018-02-09 16:00:00,2018-02-10 00:00:00+00:00
3,1518307200,15140.0,17800.0,14720.0,17200.0,5.041167e+14,7.963600e+10,Binance,2018-02-10 16:00:00,2018-02-11 00:00:00+00:00
4,1518393600,14610.0,15410.0,14580.0,15140.0,3.265759e+14,4.865200e+10,Binance,2018-02-11 16:00:00,2018-02-12 00:00:00+00:00


# HOURLY

In [3]:
dir_prefix = "../../Coin_Data/Hourly"
cnt = 0
fin_vars = ['open','high','low','close','volumefrom','volumeto']
exclude_vars = ['time','exchange','date','date_utc']
for filename in os.listdir(dir_prefix): 
    if 'at' in filename and '.pkl'  in filename:
        cnt+=1
        d = pd.read_pickle(dir_prefix + '/'+filename)
#         print(d.loc[2,fin_vars])
        
        #Convert financial features into Satoshi
        for var in fin_vars:
            d[var] = d[var].apply(lambda x: x*1e8 if x >= 1e-10 else x*1e8 + 1)
#         print(d.loc[2,fin_vars], "\n")
        
        #Save the files in the processed dataset
        pd.to_pickle(d, "../../Coin_Data/Hourly/Processed_Data/Sat_"+filename)
    

In [4]:
test = pd.read_pickle("../../Coin_Data/Hourly/Processed_Data/Sat_Hourly_ADX_at_Binance.pkl")

In [5]:
test.head()

,time,close,high,low,open,volumefrom,volumeto,exchange,date,date_utc
0,1519488000,15090.0,15150.0,14790.0,14860.0,2.060900e+13,3.081000e+09,Binance,2018-02-24 08:00:00,2018-02-24 16:00:00+00:00
1,1519491600,15040.0,15200.0,15000.0,15090.0,1.541570e+13,2.321000e+09,Binance,2018-02-24 09:00:00,2018-02-24 17:00:00+00:00
2,1519495200,15040.0,15290.0,15020.0,15040.0,1.842830e+13,2.786000e+09,Binance,2018-02-24 10:00:00,2018-02-24 18:00:00+00:00
3,1519498800,14920.0,15110.0,14830.0,15040.0,1.607100e+13,2.406000e+09,Binance,2018-02-24 11:00:00,2018-02-24 19:00:00+00:00
4,1519502400,14780.0,14920.0,14690.0,14920.0,1.553490e+13,2.301000e+09,Binance,2018-02-24 12:00:00,2018-02-24 20:00:00+00:00
